In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import RobustScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping

2025-06-19 14:19:42.045858: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-19 14:19:42.081317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750317582.103601    9509 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750317582.111319    9509 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750317582.137279    9509 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
df = pd.read_csv("data_normal.csv")
df.head()

,times,Shift,Status,Suhu Sealing Vertikal Bawah (oC),Suhu Sealing Vertical Atas (oC),Suhu Sealing Horizontal Depan/Kanan (oC),Suhu Sealing Horizontal Belakang/Kiri (oC ),Counter Output (pack),Counter Reject (pack),Speed(rpm),...,Sealing Enable,Machine Alarm,Downtime_sec,Output Time_sec,Total Time_sec,delta_Downtime_sec,delta_Output Time_sec,delta_Total Time_sec,Condition,delta_reject
0,2024-11-12 10:53:00,1,2,234.1,234.0,212.6,212.6,21036,492,75.4,...,1,1,0.0,2905.0,14306.0,0.0,80.0,80.0,0,0.0
1,2024-11-12 10:53:45,1,2,234.1,234.0,212.8,212.7,21486,492,75.4,...,0,0,0.0,2949.0,14349.0,0.0,44.0,43.0,0,0.0
2,2024-11-12 10:54:50,1,2,234.4,234.0,213.1,213.1,21930,492,75.5,...,0,0,0.0,3008.0,14411.0,0.0,59.0,62.0,0,0.0
3,2024-11-12 10:55:45,1,2,234.1,234.1,213.1,213.2,22404,492,75.5,...,1,1,0.0,3065.0,14466.0,0.0,57.0,55.0,0,0.0
4,2024-11-12 10:56:57,1,2,234.1,234.1,213.0,213.2,22836,492,75.6,...,0,0,0.0,3132.0,14536.0,0.0,67.0,70.0,0,0.0


In [4]:
df.columns

Index(['times', 'Shift', 'Status', 'Suhu Sealing Vertikal Bawah (oC)',
       'Suhu Sealing Vertical Atas (oC)',
       'Suhu Sealing Horizontal Depan/Kanan (oC)',
       'Suhu Sealing Horizontal Belakang/Kiri (oC )', 'Counter Output (pack)',
       'Counter Reject (pack)', 'Speed(rpm)', 'Availability(%)',
       'Performance(%)', 'Quality(%)', 'OEE(%)', 'Jaws Position',
       'Knife Position', 'Pump Position Stop', 'Doser Drive Enable',
       'Sealing Enable', 'Machine Alarm', 'Downtime_sec', 'Output Time_sec',
       'Total Time_sec', 'delta_Downtime_sec', 'delta_Output Time_sec',
       'delta_Total Time_sec', 'Condition', 'delta_reject'],
      dtype='object')

In [ ]:
features = [
    'Suhu Sealing Vertikal Bawah (oC)',
       'Suhu Sealing Vertical Atas (oC)',
       'Suhu Sealing Horizontal Depan/Kanan (oC)',
       'Suhu Sealing Horizontal Belakang/Kiri (oC )', 'Counter Output (pack)',
       'Counter Reject (pack)', 'Speed(rpm)', 'Availability(%)',
       'Performance(%)', 'Quality(%)', 'OEE(%)', 'Jaws Position',
       'Doser Drive Enable','Sealing Enable', 'Machine Alarm'
]

